In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
import warnings
warnings.filterwarnings("ignore")
from sklearn.linear_model import MultiTaskLasso
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score,accuracy_score,classification_report
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression,Ridge,Lasso,LogisticRegression
from datetime import datetime
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

In [2]:
df = pd.read_csv('LoanData.csv', low_memory=False)

In [3]:
data = pd.read_csv('Bondora_preprocessed2.csv')

In [4]:
index_loandate_map = df['LoanDate'].to_dict()

data['Loandate'] = data['Unnamed: 0'].map(index_loandate_map)

index_loandate_map = df['MaturityDate_Last'].to_dict()

data['MaturityDate'] = data['Unnamed: 0'].map(index_loandate_map)

In [5]:
data.drop(['Unnamed: 0'],axis=1,inplace=True)

### Loan Tenure

In [6]:
# Parse loan date and maturity date
data['Loandate'] = pd.to_datetime(data['Loandate'])
data['MaturityDate'] = pd.to_datetime(data['MaturityDate'])

# Calculate loan tenure
data['LoanTenure'] = (data['MaturityDate'].dt.year - data['Loandate'].dt.year)*12+(data['MaturityDate'].dt.month - data['Loandate'].dt.month)

# Print the loan tenure
data['LoanTenure']

0        12
1         1
2        59
3        15
4        12
         ..
77389    61
77390    49
77391    61
77392    67
77393    67
Name: LoanTenure, Length: 77394, dtype: int64

### Preferred EMI

emi = P x R x (1+R)^N / [(1+R)^N-1]<br>
“P” is the principal loan amount, <br>
“N” in tenure in months, <br>
and “R” is the prevailing interest rate.<br>

P -> Amount<br>
R -> Interest/12*100<br>
N -> LoanDuration<br>

In [7]:
data[['Amount','Interest','LoanTenure']]

,Amount,Interest,LoanTenure
0,115.0408,30.00,12
1,140.6057,25.00,1
2,319.5409,25.00,59
3,57.5205,45.00,15
4,319.5436,30.00,12
...,...,...,...
77389,3000.0000,31.01,61
77390,3000.0000,18.25,49
77391,2500.0000,24.83,61
77392,3000.0000,17.74,67


In [8]:
P = data['Amount']
R = data['Interest']/100
N = data['LoanTenure']
data['EMI'] = P * R * (1+R)**N /((1+R)**N-1)

## Eligible Loan Amount(ELA)

Components of ELA:<br>
A: “AppliedAmount”<br>
R: “Interest”(Interest rate)<br>
N: “LoanTenure”<br>
I: “IncomeTotal”<br>
L: “LiabilitiesTotal”<br>

For each row in the dataset:<br>
1- Calculate: Total Payment Due = (A + (A*r) * n<br>
2- Calculate: Max allowable amount = (I – L) * 30%<br>
3- If ( Total Payment Due <= Max allowable amount)<br>
Then ELA = AppliedAmount<br>
Else ELA = Max allowable amount<br>

In [9]:
A = data['AppliedAmount']
R = data['Interest']/100
N = data['LoanTenure']
I = data['IncomeTotal']
L = data['LiabilitiesTotal']

TotalDue = A+A*R*N
MaxAllowed = (I-L)*0.3
data['ELA'] = np.where(TotalDue < MaxAllowed, TotalDue, MaxAllowed)

## Preferred ROI(PROI)

ROI = Net income / Cost of investment x 100<br>
Cost of investment -> Total Amount.<br>
Net return -> Amount with interest<br>
Net return = AmountxInterest(This will be the net profit in addition to the Amount)

In [10]:
data['InterestAmount'] = (data['Amount']*(data['Interest']/100))
data['TotalAmount'] = (data['InterestAmount'] + data['Amount'])
data['ROI'] = (data['InterestAmount'] / data['TotalAmount'])*100

data['PROI'] = data['ROI'].median()

for i in range(data.shape[0]):
    if data['LoanTenure'].loc[i] <= 19:
        data['PROI'].loc[i] = data['PROI'].loc[i] - 5
    elif data['LoanTenure'].loc[i] > 25:
        data['PROI'].loc[i] = data['PROI'].loc[i] + 5

    if (data['AppliedAmount'].loc[i] <= 1175) & (data['AppliedAmount'].loc[i] >= 850):
        data['PROI'].loc[i] = data['PROI'].loc[i] - 5
    elif data['AppliedAmount'].loc[i] > 2000:
        data['PROI'].loc[i] = data['PROI'].loc[i] + 5

    if data['IncomeTotal'].loc[i] <= 1000:
        data['PROI'].loc[i] = data['PROI'].loc[i] - 5

    if data['DebtToIncome'].loc[i] == 0:
        data['PROI'].loc[i] = data['PROI'].loc[i] - 5
    else:
        data['PROI'].loc[i] = data['PROI'].loc[i] + 5

Target Variables<br>
- Eligible Loan Amount(ELA)
- Preferred Return On Investment(PROI)
- Equated Monthly Installments(EMI)

In [11]:
# Handling with infinite values
mask = np.isinf(data['EMI'])
data = data.drop(data[mask].index)
data.shape

(75915, 49)

In [12]:
data.columns

Index(['BidsPortfolioManager', 'BidsApi', 'BidsManual', 'NewCreditCustomer',
       'VerificationType', 'LanguageCode', 'Age', 'Gender', 'Country',
       'AppliedAmount', 'Amount', 'Interest', 'LoanDuration', 'MonthlyPayment',
       'City', 'UseOfLoan', 'Education', 'MaritalStatus', 'EmploymentStatus',
       'EmploymentDurationCurrentEmployer', 'OccupationArea',
       'HomeOwnershipType', 'IncomeTotal', 'ExistingLiabilities',
       'LiabilitiesTotal', 'RefinanceLiabilities', 'DebtToIncome', 'FreeCash',
       'MonthlyPaymentDay', 'Rating', 'Restructured', 'PrincipalPaymentsMade',
       'InterestAndPenaltyPaymentsMade', 'PrincipalBalance',
       'InterestAndPenaltyBalance', 'NoOfPreviousLoansBeforeLoan',
       'AmountOfPreviousLoansBeforeLoan', 'PreviousRepaymentsBeforeLoan',
       'PreviousEarlyRepaymentsCountBeforeLoan', 'Status_new', 'Loandate',
       'MaturityDate', 'LoanTenure', 'EMI', 'ELA', 'InterestAmount',
       'TotalAmount', 'ROI', 'PROI'],
      dtype='object')

In [13]:
imp_features = pd.read_csv('Important_features.csv')
imp_features

,InterestAndPenaltyBalance,PrincipalPaymentDifference,PrincipalPaymentsMade,PrincipalBalance,InterestAndPenaltyPaymentsMade,Interest,MonthlyPayment,Amount,AppliedAmount,LanguageCode,Rating,Country,Restructured,LoanDuration,MonthlyPaymentDay,Status_new
0,0.00,115.0408,115.0408,0.00,20.4222,30.00,130.393314,115.0408,319.5582,0,0,0,False,12,25,0
1,0.00,140.6057,140.6057,0.00,2.0227,25.00,130.393314,140.6057,191.7349,0,0,0,False,1,15,0
2,414.07,86.8409,203.1909,116.35,59.7626,25.00,130.393314,319.5409,319.5582,0,0,0,True,20,25,1
3,0.00,57.5205,57.5205,0.00,18.7323,45.00,130.393314,57.5205,127.8233,0,0,0,False,15,15,0
4,0.00,319.5436,319.5436,0.00,220.4200,30.00,130.393314,319.5436,319.5582,0,0,0,False,12,25,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77389,0.00,2308.1900,2308.1900,0.00,0.0000,31.01,107.680000,3000.0000,3000.0000,3,3,1,False,60,1,1
77390,0.00,3000.0000,3000.0000,0.00,1990.0300,18.25,85.330000,3000.0000,3000.0000,3,6,1,False,60,7,0
77391,1227.98,1239.5200,1869.7600,630.24,199.3300,24.83,80.420000,2500.0000,2500.0000,3,4,1,False,60,1,1
77392,0.00,2257.5400,2257.5400,0.00,561.5300,17.74,84.510000,3000.0000,3000.0000,0,6,0,True,60,1,1


In [96]:
X = data[['InterestAndPenaltyBalance',
       'PrincipalPaymentsMade', 'PrincipalBalance', 'Interest', 'MonthlyPayment',
       'Amount', 'AppliedAmount', 'LanguageCode', 'Rating', 'Country',
       'Restructured', 'LoanDuration', 'MonthlyPaymentDay','LiabilitiesTotal','IncomeTotal','InterestAndPenaltyPaymentsMade']]
y = data[['EMI','ELA','PROI','Status_new']]

In [97]:
for colname in X.select_dtypes(["object"]):
    X[colname], _ = X[colname].factorize()

In [98]:
X

,InterestAndPenaltyBalance,PrincipalPaymentsMade,PrincipalBalance,Interest,MonthlyPayment,Amount,AppliedAmount,LanguageCode,Rating,Country,Restructured,LoanDuration,MonthlyPaymentDay,LiabilitiesTotal,IncomeTotal,UseOfLoan,InterestAndPenaltyPaymentsMade
0,0.00,115.0408,0.00,30.00,130.393314,115.0408,319.5582,0,0,0,False,12,25,0.00,10500.0,0,20.4222
1,0.00,140.6057,0.00,25.00,130.393314,140.6057,191.7349,0,0,0,False,1,15,0.00,10800.0,0,2.0227
2,414.07,203.1909,116.35,25.00,130.393314,319.5409,319.5582,0,0,0,True,20,25,0.00,7000.0,1,59.7626
3,0.00,57.5205,0.00,45.00,130.393314,57.5205,127.8233,0,0,0,False,15,15,0.00,11600.0,2,18.7323
4,0.00,319.5436,0.00,30.00,130.393314,319.5436,319.5582,0,0,0,False,12,25,0.00,6800.0,3,220.4200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77389,0.00,2308.1900,0.00,31.01,107.680000,3000.0000,3000.0000,3,3,1,False,60,1,500.00,1400.0,2,0.0000
77390,0.00,3000.0000,0.00,18.25,85.330000,3000.0000,3000.0000,3,6,1,False,60,7,1777.46,2265.0,1,1990.0300
77391,1227.98,1869.7600,630.24,24.83,80.420000,2500.0000,2500.0000,3,4,1,False,60,1,1350.00,2500.0,2,199.3300
77392,0.00,2257.5400,0.00,17.74,84.510000,3000.0000,3000.0000,0,6,0,True,60,1,732.05,540.0,0,561.5300


In [99]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)

### Regression pipelining

In [100]:
y_train_reg = y_train.iloc[:,:3]
y_test_reg = y_test.iloc[:,:3]

y_train_clf = y_train.iloc[:,3]
y_test_clf = y_test.iloc[:,3]

In [101]:
pipeline_reg = Pipeline([
    ('stdscaler', StandardScaler()),
    ('pca', PCA()),
    ('poly', PolynomialFeatures(degree=2)),
    ('regressor', LinearRegression())
])

# fit and transform the pipeline
pipeline_reg.fit(X_train, y_train_reg)

# predict using the pipeline
pred_reg = pipeline_reg.predict(X_test)

In [102]:
print('R2_score : ', r2_score(y_test_reg, pred_reg)*100, '%')

R2_score :  87.78238820644687 %


### Classification pipelining

In [45]:
pipeline_clf = Pipeline([
    ('stdscaler', StandardScaler()),
    ('pca', PCA()),
    ('classifier', XGBClassifier())
])

In [46]:
pipeline_clf.fit(X_train,y_train_clf)
pred_clf = pipeline_clf.predict(X_test)
accuracy_score(y_test_clf,pred_clf)

0.9011394322597642

In [47]:
pickle.dump(pipeline_clf, open('pipeline_class.pkl', 'wb'))
pickle.dump(pipeline_reg, open('pipeline_reg.pkl', 'wb'))